In [48]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

In [49]:
# Wczytanie danych
df = pd.read_csv('final_dataset_undersampled.csv')

In [50]:
# Konwersja kolumny Date z obsługą błędów oraz sortowanie według daty
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%y', errors='coerce', dayfirst=True)
df = df.dropna(subset=['Date'])  # Usuwamy wiersze z niepoprawnymi datami
df = df.sort_values('Date')

In [51]:
# Przygotowanie danych:
# - FTR to wynik meczu (zmienna docelowa)
# - Usuwamy kolumny, które nie są dostępne przed rozpoczęciem meczu lub mogą powodować problemy:
#   "Unnamed: 0", "FTHG", "FTAG", "MW", "HTFormPtsStr", "ATFormPtsStr", "Date"
df_model = df.drop(columns=['Unnamed: 0', 'FTHG', 'FTAG', 'MW', 'HTFormPtsStr', 'ATFormPtsStr'])
y = df_model['FTR']
X = df_model.drop(columns=['FTR', 'Date'])


In [52]:
# Podział danych na zbiór treningowy i testowy oparty na czasie (80% najwcześniejszych, 20% najpóźniejszych)
train_size = int(0.8 * len(X))
X_train, X_test = X.iloc[:train_size].copy(), X.iloc[train_size:].copy()
y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]

In [53]:
# Przekształcenie etykiet na postać numeryczną
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc = le.transform(y_test)
print(classification_report(y_test_enc, y_pred))

              precision    recall  f1-score   support

           0       0.35      0.43      0.39       167
           1       0.54      0.42      0.47       190
           2       0.55      0.57      0.56       198

    accuracy                           0.47       555
   macro avg       0.48      0.47      0.47       555
weighted avg       0.49      0.47      0.48       555



In [54]:
# Definicja cech kategorycznych i numerycznych
categorical_features = ['HomeTeam', 'AwayTeam']
numerical_features = [col for col in X_train.columns if col not in categorical_features]

In [55]:
# Konwersja kolumn numerycznych do typu liczbowego (wartości niekonwertowalne stają się NaN)
for col in numerical_features:
    X_train[col] = pd.to_numeric(X_train[col], errors='coerce')
    X_test[col] = pd.to_numeric(X_test[col], errors='coerce')

In [56]:
# Pipeline przetwarzania danych:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

In [57]:
# Budowa pipeline'u: preprocessing + XGBoost (XGBClassifier)
model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(
        use_label_encoder=False,
        eval_metric='logloss',
        random_state=42,
        colsample_bytree=0.7,
        learning_rate=0.01,
        max_depth=3,
        n_estimators=300,
        subsample=1.0
    ))
])

In [58]:
# Trenowanie modelu
model.fit(X_train, y_train_enc)

C:\Users\dawid\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xgboost\core.py:158: UserWarning: [21:56:02] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['HTGS', 'ATGS', 'HTGC',
                                                   'ATGC', 'HTP', 'ATP', 'HM1',
                                                   'HM2', 'HM3', 'HM4', 'HM5',
                                                   'AM1', 'AM2', 'AM3', 'AM4',
                                                   'AM5', 'HTFormPts',
                                                   'ATFormPts', 'HTWinStreak3',
                                                   'HTWinStreak5',
                                                   'HTLossStreak3',
                                                   'HTLossStreak5',
                                                   '...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.01,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=3, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=300, n_jobs=None,
                               num_parallel_tree=None,
                               objective='multi:softprob', ...))])

In [59]:
# Predykcja na zbiorze testowym
y_pred_enc = model.predict(X_test)

In [60]:
# Ocena modelu
accuracy = accuracy_score(y_test_enc, y_pred_enc)
print("Dokładność modelu XGBoost:", accuracy)
print("Raport klasyfikacji:\n", classification_report(y_test_enc, y_pred_enc))

Dokładność modelu XGBoost: 0.43783783783783786
Raport klasyfikacji:
               precision    recall  f1-score   support

           0       0.34      0.40      0.36       167
           1       0.49      0.39      0.43       190
           2       0.50      0.52      0.51       198

    accuracy                           0.44       555
   macro avg       0.44      0.43      0.44       555
weighted avg       0.45      0.44      0.44       555



In [61]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from xgboost import XGBClassifier

# Lżejsza siatka hiperparametrów do XGBoost
param_grid = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__max_depth': [3, 5],
    'classifier__learning_rate': [0.01, 0.05, 0.1],
    'classifier__subsample': [0.7, 1.0],
    'classifier__colsample_bytree': [0.7, 1.0]
}
# GridSearchCV na pipeline
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='balanced_accuracy',
    cv=5,
    verbose=2,
    n_jobs=-1
)

# Dopasowanie do danych treningowych
grid_search.fit(X_train, y_train_enc)

# Najlepsze parametry i wynik walidacji
print("Najlepsze parametry:", grid_search.best_params_)
print("Najlepszy wynik walidacji (accuracy):", grid_search.best_score_)

# Predykcja i ewaluacja na zbiorze testowym
y_pred_enc = grid_search.best_estimator_.predict(X_test)
y_pred_labels = le.inverse_transform(y_pred_enc)

# Raport klasyfikacji
print("Raport klasyfikacji na zbiorze testowym:")
print(classification_report(y_test, y_pred_labels))


Fitting 5 folds for each of 72 candidates, totalling 360 fits


C:\Users\dawid\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xgboost\core.py:158: UserWarning: [21:56:22] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Najlepsze parametry: {'classifier__colsample_bytree': 0.7, 'classifier__learning_rate': 0.01, 'classifier__max_depth': 3, 'classifier__n_estimators': 300, 'classifier__subsample': 1.0}
Najlepszy wynik walidacji (accuracy): 0.452523187466449
Raport klasyfikacji na zbiorze testowym:
              precision    recall  f1-score   support

           D       0.34      0.40      0.36       167
           H       0.49      0.39      0.43       190
          NH       0.50      0.52      0.51       198

    accuracy                           0.44       555
   macro avg       0.44      0.43      0.44       555
weighted avg       0.45      0.44      0.44       555

